In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import plotly.graph_objects as go
import pandas as pd
import plotly.io as pio

In [ ]:
def sird_model(t, y, beta, gamma, alpha, n):
    """ Solves system using the solve_ivp definition and function
    
    Arguments
        y: List of variable solutions
        t: List of time steps to evaluate at
        beta: Infection rate constant
        gamma: Recovery rate constant
        alpha: Death rate
        n: Total number in the population
    
    Returns:
        ds: Change in S 
        di: Change in I
        dr: Change in R
        
    """
    s, i, r, d = y
    ds = - beta * s * i / n
    di = beta * s * i / n - gamma * i
    dr = (1 - alpha) * gamma * i
    dd = alpha * gamma * i

    return ds, di, dr, dd

In [ ]:
### Starting Values
num_tsteps = 365*2
t = np.linspace(0,365,num_tsteps)
n = 330000000
y0 = [n - 1, 1, 0, 0] # Start with a single infected individual

gamma = 0.1 # 10 Day Infectious Period
beta = 0.5 # Equivalent 5.0 R0
alpha = 0.03 # 3% Mortality Rate

infectious_time = round(1/gamma,2)
latent_time = round(1/sigma,2)

### Define steps for animation
beta_start = 0.01
beta_stop = 1
beta_step = 0.01
num_beta = int((beta_stop - beta_start)/beta_step + 1)
beta_list = np.linspace(beta_start, beta_stop, num_beta)

### DataFrame Initialization
column_names = ['beta', 'S', 'I', 'R', 'D']
data = pd.DataFrame(columns= column_names)

In [ ]:
### Pre-generate all data
for beta in beta_list:
    sird_solver = solve_ivp(sird_model, [0, 365], y0, args=(beta, gamma, alpha, n), dense_output=True)
    sird_solution = sird_solver.sol(t)
    temp_df = pd.DataFrame(list(zip([beta] * num_tsteps, sird_solution[0,:], sird_solution[1,:], sird_solution[2,:], sird_solution[3,:])), columns=column_names)
    data = pd.concat([data, temp_df])

In [ ]:
def gen_trace_list(column, trace_name, trace_color):
    trace = [go.Scatter(x=t, y=data[data['beta'] == beta][column],
                        visible=False, name = trace_name,
                        line_color = trace_color)
             for beta in data['beta'].unique()]
    return trace

s_list = gen_trace_list('S', 'Susceptible', 'red')
i_list = gen_trace_list('I', 'Infected', 'green')
r_list = gen_trace_list('R', 'Recovered', 'blue')
d_list = gen_trace_list('D', 'Dead', 'goldenrod')

starting_beta = np.where(beta_list == 0.5)[0][0]
s_list[starting_beta]['visible'] = True
i_list[starting_beta]['visible'] = True
r_list[starting_beta]['visible'] = True
d_list[starting_beta]['visible'] = True

In [ ]:
# Add all of the data to a figure
fig = go.Figure(s_list + i_list + r_list + d_list)

# Generating all of the steps
steps = []
for i in range(num_beta):
    # Define the steps
    step = dict(
        method = 'update',
        label =  str(round(beta_list[i], 2)),
        args = [{'visible': [False] * len(fig.data)},
                {"title" : "SIRD Model – Infectious Period = 10 Days (𝛾 = 0.1) | Mortality Rate (𝛼) = 3%"}]
    )
    
    # Update the visible traces for each step
    step['args'][0]['visible'][i] = True
    step['args'][0]['visible'][i+num_beta] = True
    step['args'][0]['visible'][i+2*num_beta] = True
    step['args'][0]['visible'][i+3*num_beta] = True
    
    steps.append(step)

# Generate slider
sliders = [dict(steps = steps,
                active = starting_beta,
                currentvalue={'prefix' : "β = "},
                pad = {"t" : 50})]

# Final Figure generation
fig.update_layout(sliders=sliders,
                  title = "SIRD Model – Infectious Period = 10 Days (𝛾 = 0.1) | Mortality Rate (𝛼) = 3%",
                  template='plotly_white')
fig.update_yaxes(title='Number of Individuals')
fig.update_xaxes(title='Days')

# Figure preview
fig.show()

In [ ]:
# Write figure to HTML
pio.write_html(fig, "sird-graph.html", auto_open=False)